In [1]:
from tensorscaling import scale, unit_tensor, random_tensor, marginal
import numpy as np

# Tensor scaling

Scale 3x3x3 unit tensor to certain non-uniform marginals:

In [2]:
shape = [3, 3, 3]
targets = [(.5, .25, .25), (.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-4)
res

Result(success=True, iterations=90, max_dist=9.229270866576476e-05, ..., log_cap=-0.42913652471182406)

We can also access the scaling matrices and the final scaled state:

In [3]:
print(res.gs[0], "\n")
print(res.gs[1], "\n")
print(res.gs[2])

[[ 1.55167231+0.j          0.        +0.j          0.        +0.j        ]
 [-0.14879748-0.26634419j  0.53649948+0.j          0.        +0.j        ]
 [-0.11807643+0.29571447j -0.06597004+0.17087674j  0.88592878+0.j        ]] 

[[ 0.62139109+0.j          0.        +0.j          0.        +0.j        ]
 [ 0.80160157-1.67042409j  1.80089944+0.j          0.        +0.j        ]
 [-1.44435985-0.76082311j -1.67473174-0.70599555j  1.81916559+0.j        ]] 

[[1.90212196+0.j         0.        +0.j         0.        +0.j        ]
 [0.71069385+0.48613808j 0.6103561 +0.j         0.        +0.j        ]
 [0.17655375-0.19106406j 0.07732156-0.0589244j  0.29642556+0.j        ]]


Let's now check that the W tensor *cannot* be scaled to uniform marginals:

In [4]:
shape = [2, 2, 2, 2]
W = np.zeros(shape)
W[1, 0, 0, 0] = W[0, 1, 0, 0] = W[0, 0, 1, 0] = W[0, 0, 0, 1] = .5
targets = [(.5, .5)] * 4

scale(W, targets, eps=1e-4, max_iterations=1000)

Result(success=False, iterations=1000, max_dist=0.10903114193597878, ..., log_cap=None)

To see more clearly what is going on, we can set the `verbose` flag:

In [5]:
res = scale(W, targets, eps=1e-4, max_iterations=10, verbose=True)

scaling tensor of shape (2, 2, 2, 2) and type float64
target spectra:
  0: (0.5, 0.5)
  1: (0.5, 0.5)
  2: (0.5, 0.5)
  3: (0.5, 0.5)
#000: max_dist = 0.35355339 @ sys = 1
#001: max_dist = 0.47140452 @ sys = 0
#002: max_dist = 0.56568542 @ sys = 3
#003: max_dist = 0.62853936 @ sys = 2
#004: max_dist = 0.58232323 @ sys = 1
#005: max_dist = 0.59305730 @ sys = 0
#006: max_dist = 0.59545834 @ sys = 3
#007: max_dist = 0.59262283 @ sys = 2
#008: max_dist = 0.59353004 @ sys = 1
#009: max_dist = 0.59357133 @ sys = 0
#010: max_dist = 0.59340661 @ sys = 3
did not converge!


We see that at each point in the algorithm, one of the marginals has Frobenius distance $\approx 0.59$ to being uniform. Indeed, we know that the entanglement polytope of the W tensor does not include the point corresponding to uniform marginals -- see [here](https://www.entanglement-polytopes.org/four_qubits) for an interactive visualization!

# Tuples of matrices and tensors

We can just as well only prescribe the desired spectra for subsystems.
Note that prescribing two out of three marginals amounts to *operator scaling*.

In [6]:
shape = [3, 3, 3]
targets = [(.4, .3, .3), (.7, .2, .1)]

res = scale(unit_tensor(3, 3), targets, eps=1e-6)
res

Result(success=True, iterations=31, max_dist=8.709524178694752e-07, ..., log_cap=-0.2205513908654608)

Indeed, the last two marginals are as prescribed, while the first marginal is arbitrary.

In [7]:
print(marginal(res.psi, 0).round(5), "\n")
print(marginal(res.psi, 1).round(5), "\n")
print(marginal(res.psi, 2).round(5))

[[0.34356+0.j      0.00852+0.01106j 0.01756-0.01817j]
 [0.00852-0.01106j 0.31051-0.j      0.00076-0.00935j]
 [0.01756+0.01817j 0.00076+0.00935j 0.34593-0.j     ]] 

[[ 0.4+0.j -0. -0.j  0. +0.j]
 [-0. +0.j  0.3-0.j  0. -0.j]
 [ 0. -0.j  0. +0.j  0.3+0.j]] 

[[ 0.7+0.j -0. +0.j  0. -0.j]
 [-0. -0.j  0.2+0.j  0. -0.j]
 [ 0. +0.j  0. +0.j  0.1-0.j]]
